# Transforms module demonstration

## Second quantization and the canonical anticommutation relations

$\newcommand{acomm}[1]{\{ #1 \}}$
$\newcommand{bra}[1]{\langle #1 \rvert}$
$\newcommand{ket}[1]{\lvert #1 \rangle}$
$\newcommand{set}[1]{\{ #1 \}}$
$\newcommand{parens}[1]{( #1 )}$
$\newcommand{bracks}[1]{[ #1 ]}$
Second quantization is a mathematical formalism used to describe systems
of fermions. The idea is that there are a number of *modes* which can be
occupied by a fermion or not. A system of $N$ fermionic modes is
described by a set of fermionic *annihilation operators*
$\set{a_p}_{p=1}^N$ satisfying the *canonical anticommutation relations*
$$\begin{aligned}
    \acomm{a_p, a_q} &= 0, \label{eq:car1} \\
    \acomm{a_p, a^\dagger_q} &= \delta_{pq}, \label{eq:car2}
  \end{aligned}$$ where $\acomm{A, B} := AB + BA$. The adjoint
$a^\dagger_p$ of an annihilation operator $a_p$ is called a *creation
operator*, and we refer to creation and annihilation operators as
fermionic *ladder operators*.

We will always work in a finite-dimensional vector space, and in this
setting, the anticommutation relations have the following consequences:

-   The operators $\set{a^\dagger_p a_p}_{p=1}^N$ commute with each
    other and have eigenvalues 0 and 1. These are called the *occupation
    number operators*.

-   There is a normalized vector $\ket{\text{vac}}$, called the *vacuum
    state*, which is a mutual 0-eigenvector of all
    the $a^\dagger_p a_p$.

-   If $\ket{\psi}$ is a 0-eigenvector of $a_p^\dagger a_p$, then
    $a_p^\dagger\ket{\psi}$ is a 1-eigenvector of $a_p^\dagger a_p$.
    This explains why we say that $a_p^\dagger$ creates a fermion in
    mode $p$.

-   If $\ket{\psi}$ is a 1-eigenvector of $a_p^\dagger a_p$, then
    $a_p\ket{\psi}$ is a 0-eigenvector of $a_p^\dagger a_p$. This
    explains why we say that $a_p$ annihilates a fermion in mode $p$.

-   $a_p^2 = 0$ for all $p$. One cannot create or annihilate a fermion
    in the same mode twice.

-   The set of $2^N$ vectors $$\begin{aligned}
            \parens{a^\dagger_1}^{i_1} \cdots \parens{a^\dagger_N}^{i_N} \ket{\text{vac}},
            \qquad i_1, \ldots, i_N \in \set{0, 1}
            \label{eq:orth_basis}
          \end{aligned}$$ are orthonormal.

-   The annihilation operators $\set{a_p}$ act on this basis as follows:
    $$\begin{aligned}
            \bra{\text{vac}} \bracks{\parens{a_N}^{j_N} \cdots \parens{a_1}^{j_1}}
            a_p 
            \bracks{\parens{a^\dagger_1}^{i_1} \cdots \parens{a^\dagger_N}^{i_N}} \ket{\text{vac}}
            =
            \delta_{j_p, 0} \delta_{i_p, 1} \prod_{q < p} \delta_{j_q, i_q} (-1)^{i_q}.
            \label{eq:action}
          \end{aligned}$$

See [here](http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf) for a derivation and discussion of these
consequences.

Let's instantiate some FermionOperators and check that they satisfy the canonical anticommutation relations. Let's also check that the occupation number operators commute.

In [2]:
from openfermion import *

a_2 = FermionOperator('2')
a_2_dag = FermionOperator('2^')
a_5 = FermionOperator('5')
a_5_dag = FermionOperator('5^')

num_2 = number_operator(5, 2)

zero = FermionOperator()
identity = FermionOperator(())

assert normal_ordered(anticommutator(a_2, a_2)) == zero
assert normal_ordered(anticommutator(a_2, a_2_dag)) == identity
assert normal_ordered(anticommutator(a_2, a_5)) == zero
assert normal_ordered(anticommutator(a_2, a_5_dag)) == zero

AssertionError: 

## Mapping fermions to qubits with transforms

To simulate a system of fermions on a quantum computer, we must choose a representation of the ladder operators on the Hilbert space of the qubits. In other words, we must designate a set of qubit operators which satisfy the canonical anticommutation relations. In OpenFermion a representation is specified by a transform function which maps fermionic operators (typically instances of FermionOperator) to qubit operators (instances of QubitOperator). If we work in a basis (like the computational basis), then each annihilation operator $a_p$ will then be associated with a matrix. There are many choices for the representation.

### The Jordan-Wigner Transform (JWT)

$$\begin{aligned}
    a_p &\mapsto \frac12 (X_p + \mathrm{i}Y_p) Z_1 \cdots Z_{p - 1} \\
    &= \parens{\ket{0}\bra{1}}_p Z_1 \cdots Z_{p - 1}.
\end{aligned}$$

Here, $X$, $Y$, and $Z$ are the Pauli matrices.

In [5]:
pauli_x = QubitOperator('X0')
pauli_y = QubitOperator('Y0')
pauli_z = QubitOperator('Z0')


print("pauli_x = \n{}".format(get_sparse_operator(pauli_x).toarray()))
print('')
print("pauli_y = \n{}".format(get_sparse_operator(pauli_y).toarray()))
print('')
print("pauli_z = \n{}".format(get_sparse_operator(pauli_z).toarray()))

print('')
print("jordan_wigner(a_2) = \n{}".format(jordan_wigner(a_2)))
print('')
print("jordan_wigner(a_2_dag) = \n{}".format(jordan_wigner(a_2_dag)))
print('')
print("jordan_wigner(a_5) = \n{}".format(jordan_wigner(a_5)))
print('')
print("jordan_wigner(a_5_dag) = \n{}".format(jordan_wigner(a_5_dag)))

pauli_x = 
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]

pauli_y = 
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]

pauli_z = 
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]

jordan_wigner(a_2) = 
0.5 [Z0 Z1 X2] +
0.5j [Z0 Z1 Y2]

jordan_wigner(a_2_dag) = 
0.5 [Z0 Z1 X2] +
-0.5j [Z0 Z1 Y2]

jordan_wigner(a_5) = 
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
0.5j [Z0 Z1 Z2 Z3 Z4 Y5]

jordan_wigner(a_5_dag) = 
0.5 [Z0 Z1 Z2 Z3 Z4 X5] +
-0.5j [Z0 Z1 Z2 Z3 Z4 Y5]


### The Bravyi-Kitaev transform

To define the Bravyi-Kitaev transform, it is easier to work with the operators

### The parity transform

$$\begin{aligned}
    a_p &\mapsto \frac12 (X_p Z_{p - 1} + \mathrm{i}Y_p) X_{p + 1} \cdots X_{N} \\
    &= \frac14 [(X_p + \mathrm{i} Y_p) (I + Z_{p - 1}) -
                (X_p - \mathrm{i} Y_p) (I - Z_{p - 1})]
               X_{p + 1} \cdots X_{N} \\
    &= [\parens{\ket{0}\bra{1}}_p \parens{\ket{0}\bra{0}}_{p - 1} -
        \parens{\ket{0}\bra{1}}_p \parens{\ket{1}\bra{1}}_{p - 1}]
       X_{p + 1} \cdots X_{N} \\
\end{aligned}$$

OpenFermion does not include a standalone function for performing the parity transform. However, there is functionality for defining new transforms specified by binary codes, and the binary code that specifies the parity transform is included in the library. See `examples/binary_code_transforms_demo.ipynb` for an explanation of this functionality.

In [2]:
n_modes = 10
def parity(fermion_operator):
    return binary_code_transform(fermion_operator, parity_code(n_modes))

print("parity(a_2) = \n{}".format(parity(a_2)))
print('')
print("parity(a_2_dag) = \n{}".format(parity(a_2_dag)))
print('')
print("parity(a_5) = \n{}".format(parity(a_5)))
print('')
print("parity(a_5_dag) = \n{}".format(parity(a_5_dag)))

parity(a_2) = 
0.5 [Z1 X2 X3 X4 X5 X6 X7 X8 X9] +
(-0-0.5j) [Y2 X3 X4 X5 X6 X7 X8 X9]

parity(a_2_dag) = 
0.5 [Z1 X2 X3 X4 X5 X6 X7 X8 X9] +
0.5j [Y2 X3 X4 X5 X6 X7 X8 X9]

parity(a_5) = 
0.5 [Z4 X5 X6 X7 X8 X9] +
(-0-0.5j) [Y5 X6 X7 X8 X9]

parity(a_5_dag) = 
0.5 [Z4 X5 X6 X7 X8 X9] +
0.5j [Y5 X6 X7 X8 X9]
